In [1]:
import sys

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


In [4]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')


Data Collected.


In [5]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [6]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)

In [7]:
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1A,Not assigned,M2A,43.648690,-79.385440
1,M3A,North York,Parkwoods,43.752935,-79.335641
2,M4A,North York,Victoria Village,43.728102,-79.311890
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
5,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
6,M8A,Not assigned,M9A,43.648690,-79.385440
7,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
8,M2B,Not assigned,M3B,43.648690,-79.385440
9,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529


In [8]:
neighbors.shape

(135, 5)

In [9]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=JGGBRN5XODTLZGJOMCSWIQMRH1JLGJKPSFR10XNB2R5U25GR&client_secret=KWRAMLK2HOJBQ2XLICLKXRU3M4HOCC1U2VG4Y4OPP5JF03QX&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of M2A
 are 43.64869, -79.38544.


{u'meta': {u'code': 200, u'requestId': u'5eb4c0bf7828ae001b221dfe'},
 u'response': {u'groups': [{u'items': [{u'reasons': {u'count': 0,
       u'items': [{u'reasonName': u'globalInteractionReason',
         u'summary': u'This spot is popular',
         u'type': u'general'}]},
      u'referralId': u'e-0-4bd461bc77b29c74a07d9282-0',
      u'venue': {u'categories': [{u'icon': {u'prefix': u'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
          u'suffix': u'.png'},
         u'id': u'4bf58dd8d48988d159941735',
         u'name': u'Trail',
         u'pluralName': u'Trails',
         u'primary': True,
         u'shortName': u'Trail'}],
       u'id': u'4bd461bc77b29c74a07d9282',
       u'location': {u'address': u'Glen Manor',
        u'cc': u'CA',
        u'city': u'Toronto',
        u'country': u'Canada',
        u'crossStreet': u'Queen St.',
        u'distance': 89,
        u'formattedAddress': [u'Glen Manor (Queen St.)',
         u'Toronto ON',
         u'Canada'],
   

In [10]:
# From the Foursquare lab in the previous module, we know that all the information is in the items key. 
# Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes):
        
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

M2A

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

M9A

Malvern, Rouge

M3B

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

M8B

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

M3C

Woodbine Heights

St. James Town

Humewood-Cedarvale

M8C

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

M3E

The Beaches

Berczy Park

Caledonia-Fairbanks

M8E

M1G

M3G

Leaside

Central Bay Street

Christie

M8G

M1H

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

M8H

M1J

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinity

M8J

M1K

Bayview Village

Downsview

The Danforth West, Riverd

In [17]:
# check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(675, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2A,43.64869,-79.38544,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M2A,43.64869,-79.38544,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M2A,43.64869,-79.38544,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M2A,43.64869,-79.38544,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M2A,43.64869,-79.38544,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [18]:
# check how many venues were returned for each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North",5,5,5,5,5,5
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Berczy Park,5,5,5,5,5,5
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",5,5,5,5,5,5
Business reply mail Processing Centre,5,5,5,5,5,5


In [19]:
# find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Coffee Shop,Health Food Store,Neighborhood,Pub
0,1,0,0,M2A,0
1,0,0,1,M2A,0
2,0,0,0,M2A,1
3,0,0,0,M2A,0
4,0,1,0,M2A,0


In [21]:

# examine the new dataframe size.

toronto_onehot.shape

(675, 5)

In [22]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Coffee Shop,Health Food Store,Pub
0,Agincourt,0.2,0.2,0.2,0.2
1,"Alderwood, Long Branch",0.2,0.2,0.2,0.2
2,"Bathurst Manor, Wilson Heights, Downsview North",0.2,0.2,0.2,0.2
3,Bayview Village,0.2,0.2,0.2,0.2
4,"Bedford Park, Lawrence Manor East",0.2,0.2,0.2,0.2
5,Berczy Park,0.2,0.2,0.2,0.2
6,"Birch Cliff, Cliffside West",0.2,0.2,0.2,0.2
7,"Brockton, Parkdale Village, Exhibition Place",0.2,0.2,0.2,0.2
8,Business reply mail Processing Centre,0.2,0.2,0.2,0.2
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.2,0.2,0.2,0.2


In [23]:
# confirm the new size

toronto_grouped.shape

(131, 5)

In [24]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Alderwood, Long Branch
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bayview Village
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bedford Park, Lawrence Manor East
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Berczy Park
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3         

               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3C
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3E
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3G
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3S
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3T
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----M3V
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Stor

               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Runnymede, Swansea
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Runnymede, The Junction North
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----St. James Town
----
               venue  freq
0              Trail   0.2
1        Coffee Shop   0.2
2  Health Food Store   0.2
3                Pub   0.2


----St. James Town, Cabbagetown
----
               venue  freq
0              Trail   0.2
1        Coffe

In [25]:
# sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:

num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Pub,Health Food Store,Coffee Shop
1,"Alderwood, Long Branch",Pub,Health Food Store,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Pub,Health Food Store,Coffee Shop
3,Bayview Village,Pub,Health Food Store,Coffee Shop
4,"Bedford Park, Lawrence Manor East",Pub,Health Food Store,Coffee Shop


In [30]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [31]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

D:\Program Files\Anaconda_Navigator_And_python\Anaconda2\lib\site-packages\sklearn\cluster\k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighbors

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1A,Not assigned,M2A,43.648690,-79.385440,0,Pub,Health Food Store,Coffee Shop
1,M3A,North York,Parkwoods,43.752935,-79.335641,0,Pub,Health Food Store,Coffee Shop
2,M4A,North York,Victoria Village,43.728102,-79.311890,0,Pub,Health Food Store,Coffee Shop
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Health Food Store,Coffee Shop
4,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,0,Pub,Health Food Store,Coffee Shop


In [39]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Not assigned,0,Pub,Health Food Store,Coffee Shop
1,North York,0,Pub,Health Food Store,Coffee Shop
2,North York,0,Pub,Health Food Store,Coffee Shop
3,Downtown Toronto,0,Pub,Health Food Store,Coffee Shop
4,North York,0,Pub,Health Food Store,Coffee Shop
5,Downtown Toronto,0,Pub,Health Food Store,Coffee Shop
6,Not assigned,0,Pub,Health Food Store,Coffee Shop
7,Scarborough,0,Pub,Health Food Store,Coffee Shop
8,Not assigned,0,Pub,Health Food Store,Coffee Shop
9,East York,0,Pub,Health Food Store,Coffee Shop
